In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

In [2]:
# 학습용과 평가용(제출용) 데이터를 읽어들인다.
trn = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
tst = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')

# 학습용 정답 데이터를 읽는다.
y = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')

In [4]:
IDtest = tst.custid.unique()
data = pd.concat([trn, tst], axis=0)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040039 entries, 0 to 414954
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   custid           1040039 non-null  int64 
 1   sales_month      1040039 non-null  int64 
 2   sales_day        1040039 non-null  int64 
 3   sales_dayofweek  1040039 non-null  object
 4   sales_time       1040039 non-null  int64 
 5   str_nm           1040039 non-null  object
 6   goodcd           1040039 non-null  int64 
 7   brd_nm           1040039 non-null  object
 8   corner_nm        1040039 non-null  object
 9   pc_nm            1040039 non-null  object
 10  part_nm          1040039 non-null  object
 11  team_nm          1040039 non-null  object
 12  buyer_nm         1040039 non-null  object
 13  import_flg       1040039 non-null  int64 
 14  tot_amt          1040039 non-null  int64 
 15  dis_amt          1040039 non-null  int64 
 16  net_amt          1040039 non-null  in

날짜 데이터

In [6]:
data['sales_year']=data['sales_month'].apply(lambda x: 2018 if x >12 else 2017)

In [7]:
# 월 정리
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)

In [8]:
# 주말
data['weekend'] = data['sales_dayofweek'].apply(lambda x: 1 if (x=='토' or x=='일') else 0)

In [9]:
data['sales_year']=data['sales_year'].astype('str')
data['sales_month']=data['sales_month'].astype('str')
data['sales_day']=data['sales_day'].astype('str')

In [10]:
data['date_time']=data['sales_year']+'-'+data['sales_month']+'-'+data['sales_day']

In [11]:
data['sales_year']=data['sales_year'].astype('int')
data['sales_month']=data['sales_month'].astype('int')
data['sales_day']=data['sales_day'].astype('int')

In [12]:
data['date_time'] = pd.to_datetime(data['date_time'])

In [13]:
a=data.groupby(['custid'])['date_time'].diff()

In [14]:
a=a.reset_index().set_index(data.custid)

In [15]:
a=a.reset_index()

In [16]:
a.set_index('index',inplace=True)

In [17]:
data['date_time_diff']=a.date_time

In [18]:
dtdt=data.groupby(['custid','date_time'])['date_time_diff'].sum().reset_index()

In [19]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('str')

In [20]:
dtdt.date_time_diff=dtdt.date_time_diff.apply(lambda x: x.split(' ')[0])

In [21]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('int')

In [22]:
dtdt_mean=dtdt.groupby('custid')['date_time_diff'].mean().reset_index()

# 쇼핑시간

In [23]:
dtdt_mean

,custid,date_time_diff
0,0,41.142857
1,2,18.428571
2,3,25.461538
3,4,1.500000
4,5,15.333333
...,...,...
35962,49988,28.000000
35963,49990,0.000000
35964,49992,142.000000
35965,49993,0.000000


In [24]:
ab=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_min', 'min')]).reset_index()

In [25]:
abc=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_max', 'max')]).reset_index()

In [26]:
ab['time_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [27]:
ab['time_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [28]:
ab['shop_time']=(ab['time_max']-ab['time_min'])*60+(ab['minute_max']-ab['minute_min'])

In [29]:
shop_time=ab.groupby('custid')['shop_time'].mean().reset_index()

### word2vec

In [30]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

trn2=trn.reset_index().drop('index',axis=1).copy()
trn2=trn2.reset_index()
trn2['goodcd']=trn2['goodcd'].astype('str')
train_data=list(trn2.groupby('custid')['goodcd'].unique())

In [31]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [32]:
w2v_input = oversample(train_data, 5)

In [33]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 3, min_count = 1, sg = 1)

In [34]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 21587/21587 [00:01<00:00, 14799.65it/s]


In [35]:
train_mean_vector = pd.DataFrame(train_mean_vector)
train_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.023981,0.200068,0.022746,0.008429,0.055969,-0.147475,0.124675,0.262861,-0.014822,0.087922,...,0.189596,0.036937,-0.051660,-0.002935,-0.067972,-0.143237,0.072112,-0.138113,-0.156703,0.037609
1,0.184703,0.101927,0.060066,0.157140,0.057818,-0.187687,-0.064481,0.315599,-0.144216,0.077396,...,-0.078254,0.245677,0.202411,-0.132051,-0.122044,-0.150676,-0.089461,-0.205295,-0.178355,0.144199
2,-0.291270,0.096634,0.152279,0.008337,0.062196,-0.118927,-0.058694,0.188025,-0.041053,0.123184,...,0.197973,0.058031,-0.109240,0.090842,-0.034972,0.009357,-0.069047,0.019490,-0.052048,0.101089
3,-0.055924,-0.113966,0.039334,0.162921,-0.043175,-0.368415,0.215983,0.841779,0.150075,0.002423,...,0.260856,0.219095,0.107931,-0.009410,-0.026703,0.004254,-0.119072,-0.079067,0.245670,0.005514
4,-0.136377,0.181545,0.211327,-0.034160,-0.058472,-0.183680,0.142125,0.088776,0.013789,0.111974,...,0.095392,0.162599,-0.033488,0.126233,-0.090414,-0.085551,0.098510,-0.070805,-0.037173,0.058658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.016617,0.174982,0.090610,0.102213,0.028390,-0.255650,0.162203,0.294482,-0.176366,0.130804,...,0.080021,0.143423,-0.063003,-0.238579,0.010103,-0.140052,0.021960,-0.096040,-0.091494,-0.104112
21583,-0.045056,0.126277,-0.005345,0.139266,0.013025,-0.146202,0.051776,0.347785,-0.065017,-0.079088,...,0.354890,0.073930,-0.001023,-0.142063,0.025272,0.120606,0.050755,-0.043828,-0.217618,0.045339
21584,-0.092758,0.271385,-0.005424,0.081602,-0.053663,-0.232595,-0.095151,0.278876,-0.086722,0.025125,...,0.089308,0.036489,0.352273,0.087829,-0.002012,0.011465,-0.007542,-0.108492,-0.014787,-0.192827
21585,-0.051014,-0.024733,-0.189471,0.094827,0.037206,-0.264851,0.001766,0.371398,-0.053686,0.036033,...,0.281668,0.143284,-0.014042,0.019247,-0.009690,0.088231,0.094884,-0.201303,-0.055411,0.153088


In [36]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

tst2=tst.reset_index().drop('index',axis=1).copy()
tst2=tst2.reset_index()
tst2['goodcd']=tst2['goodcd'].astype('str')
test_data=list(tst2.groupby('custid')['goodcd'].unique())

In [37]:
test_mean_vector = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector.append(tmp)
test_mean_vector = np.array(test_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 14380/14380 [00:00<00:00, 15700.02it/s]


In [38]:
test_mean_vector = pd.DataFrame(test_mean_vector)
test_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.063029,0.262608,0.023560,-0.161137,-0.181888,-0.170633,0.026449,0.208972,-0.068570,0.212443,...,0.048583,0.127265,-0.069825,-0.144089,-0.100455,-0.157868,0.041255,-0.199506,-0.088287,0.000285
1,0.011621,0.118320,0.041094,-0.014012,0.029696,-0.171390,0.024332,0.304389,0.028222,0.064186,...,0.168143,0.127393,0.041316,-0.109675,-0.103988,-0.093294,0.064802,-0.160943,-0.013352,0.002507
2,-0.154721,0.081572,0.115515,-0.065035,-0.029286,-0.121107,0.054674,0.182344,-0.044770,0.058861,...,0.317513,0.102135,0.044914,-0.201248,-0.195571,0.295452,0.174948,0.097004,-0.179747,0.145734
3,0.079099,0.206270,0.015626,0.066544,-0.069416,-0.205871,0.130075,0.351431,-0.054335,0.163187,...,0.208688,0.137542,-0.070095,-0.028317,-0.066180,0.079849,0.120892,-0.080803,-0.121721,0.025689
4,-0.013752,0.038467,0.079215,-0.043191,-0.076424,-0.254569,0.049096,0.242913,0.048321,-0.062667,...,0.311658,0.111635,0.115909,-0.134199,0.083623,0.166885,0.214158,0.131958,-0.030432,0.154663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.052513,0.127382,0.102240,-0.041510,0.003083,-0.085344,0.106123,0.149331,-0.201807,-0.012295,...,0.028441,0.146396,-0.252499,-0.001601,-0.137837,0.011168,0.140957,-0.035128,0.117698,-0.089379
14376,-0.080751,0.155199,0.045781,-0.017925,0.089158,-0.170635,-0.011217,0.231496,0.025897,-0.014048,...,0.113352,0.096250,-0.002447,0.034710,-0.075592,0.036920,0.007457,0.033633,0.003268,0.020374
14377,0.065163,0.107327,0.022211,-0.192686,0.034507,-0.187084,0.038592,0.241669,-0.171746,0.098344,...,0.331588,0.178090,-0.126120,0.100875,-0.202572,-0.087765,0.181459,0.244698,0.054240,-0.052346
14378,-0.148850,-0.000040,0.214072,0.028718,0.137039,-0.070425,0.037978,-0.028612,0.108339,-0.102126,...,0.180984,0.190317,-0.100234,-0.027959,-0.118626,0.004075,0.182223,-0.059237,-0.036283,-0.100262


In [39]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

trn2=trn.reset_index().drop('index',axis=1).copy()
trn2=trn2.reset_index()
train_data=list(trn2.groupby('custid')['brd_nm'].unique())

In [40]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [41]:
w2v_input = oversample(train_data, 5)

In [42]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 3, min_count = 1, sg = 1)

In [43]:
train_mean_vector_2 = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector_2.append(tmp)
train_mean_vector_2 = np.array(train_mean_vector_2)

100%|█████████████████████████████████████████████████████████████████████████| 21587/21587 [00:01<00:00, 13982.20it/s]


In [44]:
train_mean_vector_2 = pd.DataFrame(train_mean_vector_2)
train_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.178344,0.105850,-0.000367,0.046563,-0.275848,0.108161,0.124965,0.130908,-0.189412,-0.264588,...,0.009121,-0.111270,0.143322,-0.014480,0.042040,-0.125660,0.087444,-0.000090,-0.111664,0.221831
1,-0.159742,0.021825,0.160372,0.068815,-0.257074,0.063985,0.025633,0.065681,-0.212946,-0.208971,...,0.015741,-0.015932,0.162089,0.002160,0.019100,-0.229261,-0.067889,-0.031362,-0.046352,0.243985
2,-0.285183,0.116465,0.064869,0.234074,-0.184961,0.125919,0.012431,0.038625,-0.119666,-0.128189,...,-0.020950,-0.022836,0.193000,-0.058724,0.062418,-0.077949,-0.043073,-0.120203,-0.160464,0.144444
3,-0.290505,0.083172,0.216535,0.110587,-0.097301,0.390241,-0.002257,0.118329,-0.333057,-0.408728,...,-0.214718,0.008400,0.009828,0.050159,0.185995,-0.046572,-0.154069,-0.114660,-0.023363,-0.124461
4,-0.231829,0.034202,0.023778,0.165390,-0.201471,0.074464,0.042276,0.098801,-0.110091,-0.159074,...,-0.050114,-0.040600,0.147433,-0.079258,-0.027340,-0.181252,0.022176,0.029429,-0.192105,0.155232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.176824,0.071281,0.079347,0.101582,-0.105446,0.039295,0.026817,0.128958,-0.188847,-0.259928,...,-0.101068,-0.056316,0.196761,-0.058868,-0.024536,-0.055838,-0.080669,-0.041144,-0.154630,0.128005
21583,-0.228353,0.038323,0.112884,0.092016,-0.041935,0.038321,0.086642,0.010010,-0.153997,-0.316325,...,-0.087514,-0.163740,0.193717,-0.069007,0.128401,-0.078540,-0.130183,-0.086045,-0.077185,0.213188
21584,-0.191543,0.000382,0.120480,0.152677,-0.286981,-0.000413,0.085119,0.130209,-0.247474,-0.320288,...,0.028040,-0.064780,0.075344,0.041409,0.022361,-0.072043,-0.045915,0.054749,-0.197880,0.157196
21585,-0.101838,0.017738,0.036664,0.237069,-0.050111,0.054301,0.018627,-0.012816,-0.109608,-0.217939,...,0.048790,-0.076608,0.162124,-0.011795,0.143018,-0.102912,-0.123728,-0.005026,-0.117246,0.148187


In [45]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

tst2=tst.reset_index().drop('index',axis=1).copy()
tst2=tst2.reset_index()
test_data=list(tst2.groupby('custid')['brd_nm'].unique())

In [46]:
test_mean_vector_2 = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector_2.append(tmp)
test_mean_vector_2 = np.array(test_mean_vector_2)

100%|█████████████████████████████████████████████████████████████████████████| 14380/14380 [00:01<00:00, 13713.21it/s]


In [47]:
test_mean_vector_2 = pd.DataFrame(test_mean_vector_2)
test_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.206838,0.167460,0.000779,0.016562,-0.200798,0.147325,-0.013143,0.080544,-0.182752,-0.170307,...,-0.004497,-0.015271,0.143416,-0.001277,0.005580,-0.177626,0.018077,-0.015047,-0.065258,0.199905
1,-0.165584,0.068428,0.071313,0.135421,-0.164613,0.034201,0.058631,0.118954,-0.154230,-0.206022,...,-0.019304,-0.086523,0.129896,-0.096303,0.054745,-0.152090,-0.026188,0.019808,-0.132540,0.160860
2,-0.243817,-0.027718,0.140094,0.136712,-0.198732,0.189112,0.162729,-0.074721,-0.139128,-0.258932,...,-0.091955,-0.123720,0.120187,-0.073211,0.032595,-0.080989,-0.120131,-0.056888,-0.082997,0.138445
3,-0.207972,-0.066215,0.110911,0.226881,-0.036370,0.083503,0.114363,0.059260,-0.236970,-0.355303,...,-0.012446,-0.071517,0.202670,-0.015149,0.065785,-0.051563,-0.136461,-0.053072,0.095391,0.347672
4,-0.250290,-0.058977,0.036890,0.080139,-0.114835,0.048564,0.028253,0.035690,-0.175025,-0.166582,...,-0.034739,-0.105624,0.168097,-0.115262,-0.047710,-0.100699,-0.050854,0.051047,-0.154485,0.211251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.283718,0.078619,0.057053,0.215036,-0.348992,0.054950,0.074013,0.178097,-0.174814,-0.240574,...,-0.301440,-0.050172,0.136164,-0.197191,0.035007,-0.160246,-0.060941,0.013558,-0.311837,0.251957
14376,-0.171232,-0.129878,0.046225,0.129231,-0.283436,0.189212,0.193102,-0.014398,-0.158307,-0.252301,...,-0.010293,-0.189415,0.047693,-0.049922,-0.089928,-0.096680,-0.089367,0.252171,-0.169873,0.318054
14377,-0.157498,0.057496,0.085162,0.044260,-0.239347,0.133134,0.049829,0.254204,-0.199186,-0.092021,...,-0.084553,0.039290,0.149201,-0.268059,0.118222,-0.172335,0.034922,0.201777,-0.151267,0.188443
14378,-0.164044,-0.078153,0.083420,0.166603,-0.368248,0.148491,0.061132,0.035519,-0.087845,-0.079247,...,-0.064491,-0.041648,0.207485,-0.080112,0.037100,-0.097280,-0.154013,-0.025294,-0.200387,0.114565


In [48]:
# sales time - 아침/ 낮/ 밤 구매 나누기
data['sales_time'] = data['sales_time'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
data['sales_time_cat'] = data['sales_time'].apply(lambda x: 'mo_sales' if (9 <= x <= 12) else ('af_sales' if (13 <= x <= 18) else 'knt_sales'))

In [49]:
# dis_amt - 할인율 특성 추가
data['dis_rate'] = data['dis_amt']/data['tot_amt']

In [50]:
# 퇴근전/후 구매 빈도, 비율
def get_off_work(x):
    if 9<=x<=17:
        return('bf_work')
    else:
        return('af_work')

In [51]:
data['sales_type'] = data['sales_time'].map(get_off_work)

시간별 구매 횟수

In [52]:
freq_per_time = pd.pivot_table(index='custid',columns='sales_time',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time','tot_amt']]).reset_index()

In [53]:
sales_type = pd.pivot_table(index='custid',columns='sales_type',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','sales_type','tot_amt']]).reset_index()

In [54]:
freq_per_time.columns=['custid', '1_stf', '9_stf', "10_stf", '11_stf', '12_stf', '13_stf', '14_stf', '15_stf', '16_stf', '17_stf', '18_stf', '19_stf', '20_stf', '21_stf', '22_stf']

In [55]:
features = pd.merge(freq_per_time, sales_type, on = 'custid')

In [56]:
features = pd.merge(features, shop_time, on = 'custid')

In [57]:
features = pd.merge(features, dtdt_mean, on = 'custid')

In [58]:
def floor_transfer_a(x) :
    return  {
        "무역점" : 0,
        "본점" : 1,
        "신촌점" : 0,
        "천호점" : 1
    }.get(x)

def floor_transfer_m(x) :  
    return  {
        "무역점" : 1/10*13,
        "본점" : 1/6*13,
        "신촌점" : 1/11*13,
        "천호점" : 1
    }.get(x)
data['floor'] = (data.str_nm.apply(floor_transfer_a))*data.str_nm.apply(floor_transfer_m).astype(int)

In [59]:
features = pd.merge(features, data.groupby('custid')['floor'].mean().reset_index(), on = 'custid')

In [60]:
features['af_work_rate'] = features['af_work']/(features['af_work'] + features['bf_work'])
features['bf_work_rate'] = features['bf_work']/(features['af_work'] + features['bf_work'])

In [61]:
# 아침구매/ 낮구매/ 밤구매
freq_sales_cat = pd.pivot_table(index='custid',columns='sales_time_cat',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time_cat','tot_amt']]).reset_index()
features = pd.merge(features, freq_sales_cat,on='custid')

In [62]:
refund_freq=data.query('tot_amt<0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, refund_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'refund_freq'},inplace=True)
features.fillna(0,inplace=True)

In [63]:
sales_freq=data.query('tot_amt>0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, sales_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'sales_freq'},inplace=True)

In [64]:
# 구매 빈도 특성 추가
all_freq=data.groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, all_freq,  left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'all_freq'},inplace=True)

In [65]:
features['refund_sales_rate']=features['refund_freq']/features['sales_freq']
features['refund_all_rate'] = features['refund_freq']/features['all_freq']

방문한 날짜의 갯수

In [66]:
data_aaa=data.groupby(['custid','sales_month','sales_day'])['sales_dayofweek'].count().reset_index()

In [67]:
data_aaa.sales_dayofweek=1

In [68]:
data_aaa=data_aaa.groupby('custid')['sales_dayofweek'].sum().reset_index()

In [69]:
features = pd.merge(features, data_aaa, on = 'custid')

In [70]:
features['sales_freq_mean']=features['sales_freq']/features['sales_dayofweek']

평일에 방문하는 평균 시간

In [71]:
vuddlf=data[data.weekend==0].groupby('custid')['sales_time'].mean().reset_index()

In [72]:
features = pd.merge(features, vuddlf, how='left',on = 'custid')

In [73]:
wkdtjdgus=data[data.weekend==0].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [74]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

주말에 방문하는 평균 시간

In [75]:
wnakf=data[data.weekend==1].groupby('custid')['sales_time'].mean().reset_index()

In [76]:
features = pd.merge(features, wnakf, how='left',on = 'custid')

In [77]:
wkdtjdgus=data[data.weekend==1].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [78]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

In [79]:
# 수입상품 관련 열
sales_import=data.groupby('custid')['import_flg'].agg('sum').reset_index()
features = pd.merge(features, sales_import, on = 'custid')

features.rename(columns = {'import_flg': 'sales_import'},inplace=True)

In [80]:
# 수입상품 구매 빈도 추가
features['import_rate']=features['sales_import']/features['sales_freq']

In [81]:
# 수입상품 환불 빈도 추가
features['import_refund_rate']=features['sales_import']/(features['refund_freq']+1)

In [82]:
# 계절별 구매 빈도 추가
def f1(x):
    if 3 <= x <= 5 :
        return('spring_sales')
    elif 6 <= x <= 8 :
        return('summer_sales')
    elif 9 <= x <= 11 :
        return('fall_sales')
    else :
        return('winter_sales')    

In [83]:
data['season_sales']=data['sales_month'].apply(f1)

In [84]:
season = pd.pivot_table(data, index='custid', columns='season_sales', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features = pd.merge(features, season, how='left',on = 'custid')

In [85]:
#계절별 구매만 빈도
season_sales_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_sales_freq, how='left',on = 'custid')

In [86]:
#계절별 환불만 빈도
season_refund_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt<0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_refund_freq, how='left',on = 'custid')
features.fillna(0,inplace=True)

In [87]:
features['sales_fall_rate']=features.fall_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [88]:
features['sales_spring_rate']=features.spring_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [89]:
features['sales_summer_rate']=features.summer_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [90]:
features['sales_winter_rate']=features.winter_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [91]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [92]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [93]:
features['refund_spring_rate']=features.spring_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [94]:
features['refund_summer_rate']=features.summer_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [95]:
features['refund_winter_rate']=features.winter_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [96]:
features['all_fall_rate']=features.fall_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [97]:
features['all_spring_rate']=features.spring_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [98]:
features['all_summer_rate']=features.summer_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [99]:
features['all_winter_rate']=features.winter_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [100]:
# 주말 구매 비율
features['wd_sales_rate'] = (data.groupby('custid')['weekend'].sum().reset_index()['weekend']/features['sales_freq'])*100

월별 구매액의 합, 평균

In [101]:
month_sales_mn=data.groupby(['custid','sales_month'])['net_amt'].mean().unstack().fillna(0)

In [102]:
month_sales_mn.columns = ['mon_sale_mn'+ str(column) for column in month_sales_mn.columns]
month_sales_mn = month_sales_mn.reset_index()

In [103]:
month_sales_sm=data.groupby(['custid','sales_month'])['net_amt'].sum().unstack().fillna(0)

In [104]:
month_sales_sm.columns = ['mon_sale_sm'+ str(column) for column in month_sales_sm.columns]
month_sales_sm = month_sales_sm.reset_index()

In [105]:
features = pd.merge(features, month_sales_sm, how='left',on = 'custid')

In [106]:
features = pd.merge(features, month_sales_mn, how='left',on = 'custid')

In [107]:
# 구매 월 평균
features['month_mean'] = data.groupby('custid')['sales_month'].mean().reset_index()['sales_month']

In [108]:
# 자주 가는 월 - 원래 자주가는 
#f = pd.pivot_table(data, index='custid', columns='sales_month', values='tot_amt', 
#                   aggfunc=np.size, fill_value=0).reset_index()
f=pd.pivot_table(data[data.tot_amt>0], index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()

In [109]:
f.columns=['custid', '1_mmff', '2_mmff', '3_mmff', '4_mmff', '5_mmff', '6_mmff', '7_mmff', '8_mmff', '9_mmff', '10_mmff', '11_mmff', '12_mmff']

In [110]:
features = pd.merge(features, f, how='left',on = 'custid')

금액 관리

In [111]:
refund_net_sum=data[data.net_amt<0].groupby('custid')['net_amt'].sum().reset_index()
features = pd.merge(features, refund_net_sum, how='left',on = 'custid')

In [112]:
net_saels_all=data.groupby('custid')['net_amt'].agg([('tot_sales_all','sum'),('mean_amt_all','mean')])
features = pd.merge(features, net_saels_all, how='left',on = 'custid').fillna(0)

In [113]:
net_sales_0_all=data[data.net_amt>0].groupby('custid')['net_amt'].agg([('tot_sales','sum'),('max_sales_amt','max'),('min_sales_amt','min'),('mean_sales_amt','mean')])
features = pd.merge(features, net_sales_0_all, how='left',on = 'custid').fillna(0)

In [114]:
net_refund_all=data[data.net_amt<0].groupby('custid')['net_amt'].agg([('tot_refund','sum'),('max_refund_amt','max'),('min_refund_amt','min'),('mean_refund_amt','mean')])
features = pd.merge(features, net_refund_all, how='left',on = 'custid').fillna(0)

In [115]:
features[['mean_amt_all','mean_sales_amt','mean_refund_amt']] =round(features[['mean_amt_all','mean_sales_amt','mean_refund_amt']],2)

In [116]:
features['tot_sales_freq_mean']=features['tot_sales']/features['sales_dayofweek']

In [117]:
# 총 할인금액
features['tot_dis_amt'] = data.groupby('custid')['dis_amt'].sum().reset_index()['dis_amt']

In [118]:
# 최대, 평균 할인율
features['max_dis'] = data.groupby('custid')['dis_rate'].max().reset_index()['dis_rate']
features['mean_dis'] = data.groupby('custid')['dis_rate'].mean().reset_index()['dis_rate']

In [119]:
# 할인 받은 빈도, 비율
dis_rate=data.query("dis_rate>0").groupby('custid')['dis_rate'].count().reset_index()
features = pd.merge(features, dis_rate, how='left',on='custid').fillna(0)
features.rename(columns = {'dis_rate': 'dis_freq'},inplace=True)

In [120]:
features['dis_freq_rate'] = features['dis_freq']/features['sales_freq']

In [121]:
# 할부 빈도, 비율/ 평균 할부개월
inst_ft = data.query("inst_mon != 1").groupby('custid')['inst_mon'].agg([('inst_freq','count'),('inst_avg','mean'),('inst_max','max')])
features = pd.merge(features, inst_ft, how='left',on = 'custid').fillna(0)
features['inst_freq_rate'] = features['inst_freq']/features['sales_freq']
features['inst_freq_all_rate'] = features['inst_freq']/features['all_freq']

In [122]:
# 할부 총 달
inst_ft_sum = data.groupby('custid')['inst_mon'].agg([('inst_sum','sum')])
features = pd.merge(features, inst_ft_sum, how='left',on = 'custid').fillna(0)

In [123]:
features['tot_inst_amt']=features['tot_sales']/features['inst_sum']

In [124]:
# 할부 총 달
sales_noinst = data.groupby('custid')['inst_fee'].agg([('inst_fee_sum','sum')])
features = pd.merge(features, sales_noinst, how='left',on = 'custid').fillna(0)

In [125]:
#def inst_avg(x):
#    if x >=3:
#        return 1
#    else:
#        return 0
#
#features['inst_quan'] = features.inst_avg.apply(inst_avg)

In [126]:
# 자주가는 지점이 어딘지 원핫인코딩 한 것
data['str_nm'] = data['str_nm'].apply(lambda x: 'mu' if x=='무역점' else('bon' if x=='본점' else('cheon' if x=='천호점' else 'sin')))

In [127]:
f = data.groupby(['custid','str_nm'])['str_nm'].agg([('str_freq','count')]).reset_index()

In [128]:
lst = []
for i in tqdm_notebook(f['custid'].unique()) : 
    f2 = f.query("custid == @i")
    lst.append(f2.sort_values(by='str_freq',ascending=False)['str_nm'].iloc[0])

features['str_freq'] = lst

In [129]:
features = pd.get_dummies(features, 'str_freq')

아이디 별 매장별 몇번씩 갔는지

In [130]:
str_freq_all=data.groupby(['custid','str_nm'])['str_nm'].count().unstack().fillna(0)

In [131]:
str_freq_all.columns = ['store_'+ str(column) for column in str_freq_all.columns]
str_freq_all = str_freq_all.reset_index()

In [132]:
features = pd.merge(features, str_freq_all, how='left',on = 'custid')

매장 별로 얼마나 구매를 하였는지 총구매액

In [133]:
str_sale_sum=data[data.net_amt>0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [134]:
str_sale_sum.columns = ['str_sales'+ str(column) for column in str_sale_sum.columns]
str_sale_sum = str_sale_sum.reset_index()

In [135]:
features = pd.merge(features, str_sale_sum, how='left',on = 'custid')

In [136]:
str_refund_sum=data[data.net_amt<0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [137]:
str_refund_sum.columns = ['str_refund'+ str(column) for column in str_refund_sum.columns]
str_refund_sum = str_refund_sum.reset_index()

In [138]:
features = pd.merge(features, str_refund_sum, how='left',on = 'custid')

In [139]:
# 선호 브랜드
df = pd.merge(trn, y, on='custid')

In [140]:
twn_prefer_brd = df.query("age>=20 & age<30").brd_nm.value_counts().index[1:11].tolist()
thr_prefer_brd = df.query("age>=30 & age<40").brd_nm.value_counts().index[1:11].tolist()
for_prefer_brd = df.query("age>=40 & age<50").brd_nm.value_counts().index[1:11].tolist()
fiv_prefer_brd = df.query("age>=50 & age<60").brd_nm.value_counts().index[1:11].tolist()
six_prefer_brd = df.query("age>=60").brd_nm.value_counts().index[1:11].tolist()

In [141]:
def map_twn_brd(x):
    if x == twn_prefer_brd[0]:
        x=10
    elif x == twn_prefer_brd[1]:
        x=9
    elif x == twn_prefer_brd[2]:
        x=8
    elif x == twn_prefer_brd[3]:
        x=7
    elif x == twn_prefer_brd[4]:
        x=6
    elif x == twn_prefer_brd[5]:
        x=5
    elif x == twn_prefer_brd[6]:
        x=4
    elif x == twn_prefer_brd[7]:
        x=3
    elif x == twn_prefer_brd[8]:
        x=2
    elif x == twn_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [142]:
def map_thr_brd(x):
    if x == thr_prefer_brd[0]:
        x=10
    elif x == thr_prefer_brd[1]:
        x=9
    elif x == thr_prefer_brd[2]:
        x=8
    elif x == thr_prefer_brd[3]:
        x=7
    elif x == thr_prefer_brd[4]:
        x=6
    elif x == thr_prefer_brd[5]:
        x=5
    elif x == thr_prefer_brd[6]:
        x=4
    elif x == thr_prefer_brd[7]:
        x=3
    elif x == thr_prefer_brd[8]:
        x=2
    elif x == thr_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [143]:
def map_for_brd(x):
    if x == for_prefer_brd[0]:
        x=10
    elif x == for_prefer_brd[1]:
        x=9
    elif x == for_prefer_brd[2]:
        x=8
    elif x == for_prefer_brd[3]:
        x=7
    elif x == for_prefer_brd[4]:
        x=6
    elif x == for_prefer_brd[5]:
        x=5
    elif x == for_prefer_brd[6]:
        x=4
    elif x == for_prefer_brd[7]:
        x=3
    elif x == for_prefer_brd[8]:
        x=2
    elif x == for_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [144]:
def map_fiv_brd(x):
    if x == fiv_prefer_brd[0]:
        x=10
    elif x == fiv_prefer_brd[1]:
        x=9
    elif x == fiv_prefer_brd[2]:
        x=8
    elif x == fiv_prefer_brd[3]:
        x=7
    elif x == fiv_prefer_brd[4]:
        x=6
    elif x == fiv_prefer_brd[5]:
        x=5
    elif x == fiv_prefer_brd[6]:
        x=4
    elif x == fiv_prefer_brd[7]:
        x=3
    elif x == fiv_prefer_brd[8]:
        x=2
    elif x == fiv_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [145]:
def map_six_brd(x):
    if x == six_prefer_brd[0]:
        x=10
    elif x == six_prefer_brd[1]:
        x=9
    elif x == six_prefer_brd[2]:
        x=8
    elif x == six_prefer_brd[3]:
        x=7
    elif x == six_prefer_brd[4]:
        x=6
    elif x == six_prefer_brd[5]:
        x=5
    elif x == six_prefer_brd[6]:
        x=4
    elif x == six_prefer_brd[7]:
        x=3
    elif x == six_prefer_brd[8]:
        x=2
    elif x == six_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [146]:
data['twn_brd_point'] = data['brd_nm'].apply(map_twn_brd)
data['thr_brd_point'] = data['brd_nm'].apply(map_thr_brd)
data['for_brd_point'] = data['brd_nm'].apply(map_for_brd)
data['fiv_brd_point'] = data['brd_nm'].apply(map_fiv_brd)
data['six_brd_point'] = data['brd_nm'].apply(map_six_brd)

In [147]:
twn_brd_point=data.groupby('custid')['twn_brd_point'].mean().reset_index()
thr_brd_point=data.groupby('custid')['thr_brd_point'].mean().reset_index()
for_brd_point=data.groupby('custid')['for_brd_point'].mean().reset_index()
fiv_brd_point=data.groupby('custid')['fiv_brd_point'].mean().reset_index()
six_brd_point=data.groupby('custid')['six_brd_point'].mean().reset_index()

In [148]:
features = pd.merge(features, twn_brd_point, how='left',on='custid')
features = pd.merge(features, thr_brd_point, how='left',on='custid')
features = pd.merge(features, for_brd_point, how='left',on='custid')
features = pd.merge(features, fiv_brd_point, how='left',on='custid')
features = pd.merge(features, six_brd_point, how='left',on='custid')

In [149]:
data['buyer_nm'] = data['buyer_nm'].str.replace('행사장.*','행사장')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁A','피혁')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁B','피혁')

data['buyer_nm'] = data['buyer_nm'].map({'화장품':'a', '수입명품':'b', '캐릭터캐주얼':'c', '트래디셔널캐주얼':'d', '유아동복':'e', '니트단품':'f', '영캐주얼':'g',
       '엘레강스캐주얼':'h', '가전':'i', '섬유':'j', '장신구':'k', '조리욕실':'l', '스포츠':'m', '침구수예':'n', '피혁':'o', '일반식품':'p',
       '유니캐주얼':'q', '정장셔츠':'r', '디자이너부띠끄':'s', '문화완구':'t', '타운모피':'u', '조리식품':'v', '기타바이어':'w',
       '도자기크리스탈':'x', '가구':'y', '생활용품':'z', '행사장':'aa', '청과곡물':'bb', '점외':'cc'})

buyer = pd.pivot_table(index='custid',columns='buyer_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','buyer_nm','tot_amt']]).reset_index()
features = pd.merge(features, buyer, on='custid')

In [150]:
#구매상품군 다양성
n = data.part_nm.nunique()
f = data.groupby('custid')['part_nm'].agg([('part_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [151]:
#브랜드 다양성
n=data.brd_nm.nunique()
f = data.groupby('custid')['brd_nm'].agg([('brd_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [152]:
#매장 이용 다양성
n = 4
f = data.groupby('custid')['str_nm'].agg([('str_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [153]:
#위에서는 출근 시간 9~17시 퇴근시간 이후로 했는데 좀더 세분화
def f2(x):
    if 9 <= x < 12 :
        return('9_12_sale')
    elif 12 <= x < 14 :
        return('12_14_sale')
    elif 14 <= x < 16 :
        return('14_16_sale')
    elif 16 <= x < 18 :
        return('16_18_sale')
    else :
        return('18_sale')  

data['timeslot2'] = data.sales_time.apply(f2)

In [154]:
new_sales_type = pd.pivot_table(index='custid',columns='timeslot2',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','timeslot2','tot_amt']]).reset_index()

In [155]:
features=pd.merge(features, new_sales_type, on='custid')

In [156]:
features['12_14_sale_ratio']=features['12_14_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['14_16_sale_ratio']=features['14_16_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['16_18_sale_ratio']=features['16_18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['18_sale_ratio']=features['18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['9_12_sale_ratio']=features['9_12_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])

In [157]:
#충동지수 충동구매를 하는사람들은 누구일까? 여유가있는 나이가 많은 사람들이 충동구매를 할거같다. 곱하기대신 더하기도 생각해보기

features['chungdong_sale']=(features.max_sales_amt/features.tot_sales)
features['chungdong_sale_freq']= features.chungdong_sale*features.sales_freq

In [158]:
features['mean_min_sales']=features.mean_sales_amt+features.min_sales_amt

In [159]:
a=features[['custid','mean_min_sales']]

In [160]:
df_LEFT_JOIN = pd.merge(data, a, left_on='custid', right_on='custid', how='left')

In [161]:
df_LEFT_JOIN['chungdong']=df_LEFT_JOIN.net_amt >df_LEFT_JOIN.mean_min_sales

In [162]:
chung=df_LEFT_JOIN.groupby('custid')['chungdong'].sum().reset_index()

In [163]:
features = pd.merge(features, chung, left_on='custid', right_on='custid', how='left')

In [164]:
a=data.groupby(['custid','sales_month','sales_day','sales_dayofweek'])['custid'].size().unstack().reset_index()

In [165]:
a.iloc[:,3:]=a.iloc[:,3:]/a.iloc[:,3:]

In [166]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [167]:
features = pd.merge(features, a, on='custid')

In [168]:
features = features.rename(columns={'월':'mon_opn','화':'tue_opn','수':'wen_opn','목':'thr_opn','금':'fri_opn','토':'sat_opn','일':'sun_opn'})

In [169]:
features['mon_opn_rate']=features.mon_opn/features.sales_dayofweek
features['tue_opn_rate']=features.tue_opn/features.sales_dayofweek
features['wen_opn_rate']=features.wen_opn/features.sales_dayofweek
features['thr_opn_rate']=features.thr_opn/features.sales_dayofweek
features['fri_opn_rate']=features.fri_opn/features.sales_dayofweek
features['sat_opn_rate']=features.sat_opn/features.sales_dayofweek
features['sun_opn_rate']=features.sun_opn/features.sales_dayofweek

In [170]:
a=data[data.net_amt>0].groupby(['custid','sales_month','sales_day','sales_dayofweek'])['net_amt'].sum().unstack().reset_index()

In [171]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [172]:
features = pd.merge(features, a, on='custid')

In [173]:
features = features.rename(columns={'월':'mon_pri','화':'tue_pri','수':'wen_pri','목':'thr_pri','금':'fri_pri','토':'sat_pri','일':'sun_pri'})

In [174]:
features['mon_pri_rate']=features.mon_pri/features.tot_sales
features['tue_pri_rate']=features.tue_pri/features.tot_sales
features['wen_pri_rate']=features.wen_pri/features.tot_sales
features['thr_pri_rate']=features.thr_pri/features.tot_sales
features['fri_pri_rate']=features.fri_pri/features.tot_sales
features['sat_pri_rate']=features.sat_pri/features.tot_sales
features['sun_pri_rate']=features.sun_pri/features.tot_sales

충동구매를 하고 환불을 한 횟수

In [175]:
features['max_min_sales']=features['max_sales_amt']-features['min_sales_amt']

In [176]:
def shopfnc(m):
    if m < 5:
        return 1
    elif m < 12.94 :
        return 2 
    elif m < 21.538 :
        return 3 
    elif m < 33.909 :
        return 4
    else:
        return 5

In [177]:
features['shop_time_mean']= features.shop_time.apply(lambda x : shopfnc(x))

In [178]:
for i in np.arange(0.1,1,0.1):
    print(features.tot_sales.quantile(q=i))

249406.0
498646.0000000001
795650.0
1165370.0000000002
1622598.0
2254104.0
3107333.5999999996
4541631.800000001
7485838.200000001


In [179]:
# 10분위별로 (1 ~ 10 범위)
def fnc(m) : 
    if m < 249406.0:
        return 1
    elif m <498646 :
        return 2 
    elif m < 795650.0 :
        return 3 
    elif m < 1165370:
        return 4 
    elif m < 1622598.0 :
        return 5 
    elif m < 2254104 :
        return 6 
    elif m < 3107333:
        return 7 
    elif m < 4541631 :
        return 8
    elif m < 7485838 :
        return 9 
    else:
        return 10

features['tot_sales_amt_quan']= features.tot_sales.apply(lambda x : fnc(x))

In [180]:
for i in np.arange(0.1,1,0.1):
    print(features.max_sales_amt.quantile(q=i))

98400.0
149000.0
199000.0
261250.0
330600.0
416000.0
534850.0
736250.0
1180000.0


In [181]:
# 10분위별로 가격선호도 조사 (1 ~ 10 범위)
def fnc2(m) : 
    if m < 98400.0 :
        return 1
    elif m < 149000.0 :
        return 2 
    elif m < 199000.0 :
        return 3 
    elif m < 261250.0 :
        return 4 
    elif m < 330600.0 :
        return 5 
    elif m < 416000.0 :
        return 6 
    elif m < 534850.0 :
        return 7 
    elif m < 736250.0 :
        return 8
    elif m < 1180000.0 :
        return 9 
    else:
        return 10

features['max_amt_quan']= features.max_sales_amt.apply(lambda x : fnc2(x))

In [182]:
#아동상품 구매건수
f = data[data.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [183]:
features['baby_sales_freq']=features['baby_sales']/features.sales_freq

In [184]:
#할부 총합
sales_installment=data.groupby('custid')['inst_mon'].agg('sum').reset_index()
features = pd.merge(features, sales_installment, on='custid')

In [185]:
data['team_nm'] = data['team_nm'].map({'잡화가용팀':'aa', '의류패션팀':'bb', '식품팀':'cb', '인터넷백화점':'dd'})

In [186]:
f = data.groupby(['custid', 'team_nm'])['custid'].count().unstack()
f = f.fillna(0)
f.columns = ['Team_'+ str(column) for column in f.columns]
f = f.reset_index()

In [187]:
features = pd.merge(features, f, on = 'custid')

In [188]:
features['food_rate']=features.Team_cb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [189]:
features['cloth_rate']=features.Team_bb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [190]:
features['stuff_rate']=features.Team_aa/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [191]:
food_list=['조리식품','축산물','농산물','수산물']
t1 = data[data.net_amt>0].query("pc_nm == @food_list").groupby('custid')['net_amt'].agg([('food_sales', np.sum)])
features= pd.merge(features,t1, on = 'custid', how='left').fillna(0)
features['food_sales_rate'] = features['food_sales'] / features['tot_sales']

In [192]:
st=f"(net_amt>0) and (pc_nm=='화장품')"

In [193]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_sales', np.sum)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_sales_rate'] = features['face_goods_sales'] / features['tot_sales']

In [194]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_freq', np.size)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_goods_rate'] = features['face_goods_freq'] / features['sales_freq']

In [195]:
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부띠끄','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부틱','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('트래디셔널','트레디셔널')
data['pc_nm'] = data['pc_nm'].str.replace('침구/수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('침구,수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('피혁A','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('피혁B','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터케쥬얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터캐주얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진캐주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진케주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부틱','로얄부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부띠끄','로얄부띠크')

In [196]:
twn_trend_brd = df.query("age>=20 & age<30").pc_nm.value_counts().index[1:11].tolist()
thr_trend_brd = df.query("age>=30 & age<40").pc_nm.value_counts().index[1:11].tolist()
for_trend_brd = df.query("age>=40 & age<50").pc_nm.value_counts().index[1:11].tolist()
fiv_trend_brd = df.query("age>=50 & age<60").pc_nm.value_counts().index[1:11].tolist()
six_trend_brd = df.query("age>=60").pc_nm.value_counts().index[1:11].tolist()

def map_twn_pc(x):
    if x == twn_trend_brd[0]:
        x=10
    elif x == twn_trend_brd[1]:
        x=9
    elif x == twn_trend_brd[2]:
        x=8
    elif x == twn_trend_brd[3]:
        x=7
    elif x == twn_trend_brd[4]:
        x=6
    elif x == twn_trend_brd[5]:
        x=5
    elif x == twn_trend_brd[6]:
        x=4
    elif x == twn_trend_brd[7]:
        x=3
    elif x == twn_trend_brd[8]:
        x=2
    elif x == twn_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_thr_pc(x):
    if x == thr_trend_brd[0]:
        x=10
    elif x == thr_trend_brd[1]:
        x=9
    elif x == thr_trend_brd[2]:
        x=8
    elif x == thr_trend_brd[3]:
        x=7
    elif x == thr_trend_brd[4]:
        x=6
    elif x == thr_trend_brd[5]:
        x=5
    elif x == thr_trend_brd[6]:
        x=4
    elif x == thr_trend_brd[7]:
        x=3
    elif x == thr_trend_brd[8]:
        x=2
    elif x == thr_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_for_pc(x):
    if x == for_trend_brd[0]:
        x=10
    elif x == for_trend_brd[1]:
        x=9
    elif x == for_trend_brd[2]:
        x=8
    elif x == for_trend_brd[3]:
        x=7
    elif x == for_trend_brd[4]:
        x=6
    elif x == for_trend_brd[5]:
        x=5
    elif x == for_trend_brd[6]:
        x=4
    elif x == for_trend_brd[7]:
        x=3
    elif x == for_trend_brd[8]:
        x=2
    elif x == for_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_fiv_pc(x):
    if x == fiv_trend_brd[0]:
        x=10
    elif x == fiv_trend_brd[1]:
        x=9
    elif x == fiv_trend_brd[2]:
        x=8
    elif x == fiv_trend_brd[3]:
        x=7
    elif x == fiv_trend_brd[4]:
        x=6
    elif x == fiv_trend_brd[5]:
        x=5
    elif x == fiv_trend_brd[6]:
        x=4
    elif x == fiv_trend_brd[7]:
        x=3
    elif x == fiv_trend_brd[8]:
        x=2
    elif x == fiv_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_six_pc(x):
    if x == six_trend_brd[0]:
        x=10
    elif x == six_trend_brd[1]:
        x=9
    elif x == six_trend_brd[2]:
        x=8
    elif x == six_trend_brd[3]:
        x=7
    elif x == six_trend_brd[4]:
        x=6
    elif x == six_trend_brd[5]:
        x=5
    elif x == six_trend_brd[6]:
        x=4
    elif x == six_trend_brd[7]:
        x=3
    elif x == six_trend_brd[8]:
        x=2
    elif x == six_trend_brd[9]:
        x=1
    else:
        x=0
    return x

data['twn_pc_point'] = data['pc_nm'].apply(map_twn_pc)
data['thr_pc_point'] = data['pc_nm'].apply(map_thr_pc)
data['for_pc_point'] = data['pc_nm'].apply(map_for_pc)
data['fiv_pc_point'] = data['pc_nm'].apply(map_fiv_pc)
data['six_pc_point'] = data['pc_nm'].apply(map_six_pc)

twn_pcnm_point=data.groupby('custid')['twn_pc_point'].mean().reset_index()
thr_pcnm_point=data.groupby('custid')['thr_pc_point'].mean().reset_index()
for_pcnm_point=data.groupby('custid')['for_pc_point'].mean().reset_index()
fiv_pcnm_point=data.groupby('custid')['fiv_pc_point'].mean().reset_index()
six_pcnm_point=data.groupby('custid')['six_pc_point'].mean().reset_index()

features = pd.merge(features, twn_pcnm_point, how='left',on='custid')
features = pd.merge(features, thr_pcnm_point, how='left',on='custid')
features = pd.merge(features, for_pcnm_point, how='left',on='custid')
features = pd.merge(features, fiv_pcnm_point, how='left',on='custid')
features = pd.merge(features, six_pcnm_point, how='left',on='custid')

In [197]:
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('young_pc', lambda x: list(x).count('캐릭터정장')+list(x).count('영트랜디')+list(x).count('영캐릭터캐쥬얼')+list(x).count('트랜디 케쥬얼')+list(x).count('캐릭터캐쥬얼')+list(x).count('장신구')+list(x).count('피혁')+list(x).count('섬유')+list(x).count('소품')+list(x).count('핸드백')+list(x).count('(주)현스포츠아쌤'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [198]:
features['20goods_rate']=features['young_pc']/features.sales_freq

In [199]:
#50대선호

f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('butique_freq', lambda x: list(x).count('로얄부띠크')+list(x).count('디자이너부띠크'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [200]:
features['50goods_rate']=features['butique_freq']/features.sales_freq

In [201]:
#트랜드 명품 구매비율
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('expensive_freq', lambda x: list(x).count('명품토탈')+list(x).count('명품시즌'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [202]:
features['20~30expensive_rate']=features['expensive_freq']/features.sales_freq

In [203]:
data['pc_nm'] = data['pc_nm'].map({'화장품':'face_goods', '명품토탈':'expensive_total', '캐릭터정장':'chr_suit',  '수예':'suae', '유아/완구':'baby_wangu', '패션슈즈':'fashoin_shoes',
       '가전':'electronic', '영트랜디':'young_trend', '조리식품':'instant_food', '스포츠':'sports', '아동':'baby', '트래디셔널캐쥬얼':'trady_ca', '가전/문화':'ele_cultur', '셔츠':'shirts',
       '란제리':'rangeri', '식기':'food_case', '영캐릭터캐쥬얼':'young_chr', '골프':'golf', '캐릭터캐쥬얼':'chr_cha', '엘레강스캐쥬얼':'elegangs', '트레디셔널':'tradi',
       '정장':'suit', '일반식품':'normal_food', '아동복':'baby_cloth', '축산물':'pig', '미확인pc':'unknown_pc','농산물':'vegetable','트랜디 케쥬얼':'trendi cha',
        '영트렌디':'young_trendi','여성캐주얼':'girl_cha','문화용품':'culture_goods','생활용품':'lsang','문화':'culture','니트,단품,모피':'neat_mofi','섬유잡화':'sumyou_goods','내의란제리':'inrangeri',
        '진캐쥬얼':'jean_ca3','여성구두':'girl_shoes','명품시즌':'expensive_season','남여구두':'man_girl','장신구':'jangsingu','수산물':'susanmul','악세사리':'accesory','섬유':'sumyou',
        '어덜트캐주얼':'adult_cha','소품':'sopeum','캐주얼':'cha','미씨캐주얼':'messi_cha','영캐주얼':'young_cha','타운란제리':'townrangeri',
        '핸드백':'handbag','수입명품':'suip_goods','디자이너부띠크':'designier_beatuty','쇼핑보증':'shooping_bo','미씨캐릭터':'messi_cha5',
        '가구':'gagu','캐쥬얼':'cha2','니트/단품':'neat_one','남성잡화':'man_jobs','(주)현스포츠아쌤':'sportsam','침구수예':'chimgu4','패션시즌':'fashion_season',
        '수입의류':'suip_cloth','용기보증':'youngi','사이버쇼핑':'cyber_shopping'})

In [204]:
#주구매상품
f = data.groupby('custid')['pc_nm'].agg([
    ('main_goods', lambda x: x.value_counts().index[0])
]).reset_index()

In [205]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_goods')

In [206]:
f_p = data[data.tot_amt>0].groupby('pc_nm')['tot_amt'].agg([
    ('main_goods_price', 'mean')
]).reset_index()

In [207]:
f = pd.merge(f, f_p,  left_on = 'main_goods',right_on='pc_nm',how='left')

In [208]:
features = pd.merge(features, f[['custid','main_goods_price']],  left_on = 'custid',right_on='custid',how='left')

In [209]:
pc_count=data[data.tot_amt>0].groupby(['custid','pc_nm'])['pc_nm'].count().unstack().fillna(0)

pc_count.columns = ['pc_size_'+ str(column) for column in pc_count.columns]
pc_count = pc_count.reset_index()

features = pd.merge(features, pc_count, how='left',on = 'custid')

In [210]:

data['part_nm'] = data['part_nm'].map({'명품잡화':'aaa', '잡화파트':'bbb', '남성의류':'ccc', '골프/유니캐쥬얼':'ddd', '가정용품':'eee', '아동':'fff', '잡화':'ggg',
       '영어덜트캐쥬얼':'hhh', '영라이브':'iii', '공산품':'jjj', '스포츠캐쥬얼':'kkk', '여성캐주얼':'lll', '여성정장':'mmm', '케주얼,구두,아동':'nnn', '남성정장스포츠':'ooo', '아동문화':'ppp',
       '로얄부띠끄':'qqq', '공산품파트':'rrr', '생식품파트':'sss', '여성의류파트':'ttt', '가정용품파트':'uuu', '아동,스포츠':'vvv', '패션잡화':'www',
       '영캐릭터':'abbb', '영플라자':'abbc', '생식품':'abbd', '스포츠캐주얼':'abbe', '여성캐쥬얼':'abbf', '로얄부틱':'abbg','인터넷백화점':'abbh'})


In [211]:
#주구매상품분류
f = data.groupby('custid')['part_nm'].agg([
    ('main_part', lambda x: x.value_counts().index[0])
]).reset_index()

In [212]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part')

In [213]:
f_p = data[data.tot_amt>0].groupby('part_nm')['tot_amt'].agg([
    ('main_part_price', 'mean')
]).reset_index()

In [214]:
f = pd.merge(f, f_p,  left_on = 'main_part',right_on='part_nm',how='left')

In [215]:
features = pd.merge(features, f[['custid','main_part_price']],  left_on = 'custid',right_on='custid',how='left')

In [216]:
part_count=data[data.tot_amt>0].groupby(['custid','part_nm'])['part_nm'].count().unstack().fillna(0)

part_count.columns = ['part_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [217]:
#내점 당 구매건수 / 내점당 구매액
features['one_sales_freq']=features.sales_freq/features.sales_dayofweek
features['one_tot_amt']=features.tot_sales/features.sales_dayofweek

In [218]:
data['part_pc'] = data['part_nm'].astype(str) + "_" + data['pc_nm'].astype(str)

In [219]:
#주구매상품분류
f = data.groupby('custid')['part_pc'].agg([
    ('main_part_pc', lambda x: x.mode()[0])
]).reset_index()

In [220]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part_pc')

In [221]:
f_p = data[data.tot_amt>0].groupby('part_pc')['tot_amt'].agg([
    ('main_part_pc_price', 'mean')
]).reset_index()

In [222]:
f = pd.merge(f, f_p,  left_on = 'main_part_pc',right_on='part_pc',how='left')

In [223]:
features = pd.merge(features, f[['custid','main_part_pc_price']],  left_on = 'custid',right_on='custid',how='left')

In [224]:
part_count=data[data.tot_amt>0].groupby(['custid','part_pc'])['part_pc'].count().unstack().fillna(0)

part_count.columns = ['part_pc_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [225]:
features1=features.copy()

In [226]:
i_max=[]
for i in list(features.columns):
    if features[i].max()>50000:
        i_max.append(i)

In [227]:
i_max_min=[]
for i in i_max:
    if features[i].min()>0:
        i_max_min.append(i)

In [228]:
features[i_max_min]=np.log1p(features[i_max_min])

In [229]:
# 데이터 분할
X_train = pd.DataFrame({'custid':trn['custid'].unique()})
X_train = pd.merge(X_train, features, how='left',on='custid')

y_train = y['age']

X_test = pd.DataFrame({'custid':tst['custid'].unique()})
X_test = pd.merge(X_test, features, how='left',on='custid')

In [230]:
X_train.drop('custid',axis=1,inplace=True)
X_test.drop('custid',axis=1,inplace=True)

In [231]:
X_train.shape, X_test.shape, y_train.shape

((21587, 757), (14380, 757), (21587,))

In [232]:
X_columns=X_train.columns

In [232]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [233]:
X_train_scaled=pd.DataFrame(X_train_scaled,columns=X_columns)
X_test_scaled=pd.DataFrame(X_test_scaled,columns=X_columns)

In [233]:
X_train_scaled = pd.concat([X_train, train_mean_vector], axis=1)
X_test_scaled = pd.concat([X_test, test_mean_vector], axis=1)

In [234]:
X_train_scaled = pd.concat([X_train_scaled, train_mean_vector_2], axis=1)
X_test_scaled = pd.concat([X_test_scaled, test_mean_vector_2], axis=1)

In [235]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape

((21587, 957), (14380, 957), (21587,))

In [239]:
X_train_scaled.to_csv('hyun_features' , index = False)

In [240]:
X_test_scaled.to_csv('hyun_features_te' , index = False)